In [90]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [91]:
# Load dataset
df_titanic = pd.read_csv(r'C:\Users\marku\Desktop\ML\MLGit\datasets\titanic.csv')

In [92]:
# Sets null values equal to the median of that column
def handle_null_median(df):
    # Need to set inplace=True, so it doesn't create a copy of the dataframe. Tried without and this led to null-values not being removed
    df['Fare'].fillna(df['Fare'].median(), inplace=True)
    df['Age'].fillna(df['Age'].median(), inplace=True)

    df['Embarked'].fillna('S', inplace=True)

    return  df

df_titanic = handle_null_median(df_titanic)

In [93]:
df_titanic['Sex'] = df_titanic['Sex'].replace(['female', 'male'], [0,1])

In [94]:
df_titanic['FamilyMembersCount'] = df_titanic['Parch'] + df_titanic['SibSp'] + 1
df_titanic = df_titanic.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'], axis=1)

In [95]:
df_XGB = df_titanic.copy()
df_RF = df_titanic.copy()
df_GNB = df_titanic.copy()

In [96]:
def bins(df): # This is kind of feature engineering as well
    df['AgeGroup'] = 0
    df.loc[df['Age'] < 16.336, 'AgeGroup' ] = 0
    df.loc[(df['Age'] >= 16.336) & (df['Age'] < 32.252), 'AgeGroup'] = 1
    df.loc[(df['Age'] >= 32.252) & (df['Age'] < 48.168), 'AgeGroup'] = 2
    df.loc[(df['Age'] >= 48.168) & (df['Age'] < 64.084), 'AgeGroup'] = 3
    df.loc[df['Age'] >= 64.084, 'AgeGroup'] = 4

    # Could also create bins for fare, but not sure Fare is needed.
    return df

df_XGB = bins(df_titanic)
df_XGB

,Survived,Pclass,Sex,Age,Fare,Embarked,FamilyMembersCount,AgeGroup
0,0,3,1,22.0,7.2500,S,2,1
1,1,1,0,38.0,71.2833,C,2,2
2,1,3,0,26.0,7.9250,S,1,1
3,1,1,0,35.0,53.1000,S,2,2
4,0,3,1,35.0,8.0500,S,1,2
...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,13.0000,S,1,1
887,1,1,0,19.0,30.0000,S,1,1
888,0,3,0,28.0,23.4500,S,4,1
889,1,1,1,26.0,30.0000,C,1,1


In [97]:
df_titanic['Pclass'] =  df_titanic['Pclass'].astype('category')
df_titanic['Embarked'] =  df_titanic['Embarked'].astype('category')
df_titanic['AgeGroup'] =  df_titanic['AgeGroup'].astype('category')
df_titanic = df_titanic.drop('Age', axis=1)

In [98]:
df_XGB = pd.get_dummies(df_titanic)
df_RF = pd.get_dummies(df_titanic)
df_GNB = pd.get_dummies(df_titanic)

# XGBoost

In [99]:
train_X, test_X, train_y, test_y = train_test_split(df_XGB.drop('Survived', axis=1), df_XGB['Survived'], random_state=42)

model = XGBClassifier()

XGB_scores = cross_val_score(model, train_X, train_y)
(XGB_scores.mean(), XGB_scores.std())

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

(0.8040062843676357, 0.0326223908886716)

In [100]:
model.fit(train_X, train_y)
pred = model.predict(test_X)
accuracy_score(test_y, pred)

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.8116591928251121

In [101]:
XGBParam = {
    'max_depth': [4, 5, 6, 7],
    'subsample': [0.8, 1],
    'n_estiamtors': [100, 200],
    'eta': [.2, .3, .4]
    }

In [102]:
XGB_Grid = GridSearchCV(XGBClassifier(verbosity=0), XGBParam)
XGB_scores = cross_val_score(XGB_Grid, train_X, train_y)
(XGB_scores.mean(), XGB_scores.std())

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

(0.8129053978229155, 0.03287738840331893)

In [103]:
XGB_Grid.fit(train_X, train_y)
pred = XGB_Grid.predict(test_X)
accuracy_score(test_y, pred)

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

0.8251121076233184

# Random Forrest Classifier

In [104]:
train_X, test_X, train_y, test_y = train_test_split(df_RF.drop('Survived', axis=1), df_RF['Survived'], random_state=42)

model = RandomForestClassifier()

RF_scores = cross_val_score(model, train_X, train_y)
(RF_scores.mean(), RF_scores.std())

(0.7815396700706991, 0.02937029129685278)

In [105]:
model.fit(train_X, train_y)
pred = model.predict(test_X)
accuracy_score(test_y, pred)

0.820627802690583

In [106]:
RFParam = {
    "max_features":['auto'],
    "max_depth":[2, 6, 12, 24, 48, 96],
    "n_estimators":[50, 100, 200, 300],
    "min_samples_leaf": range(1, 10, 1)
}

RF_Grid = GridSearchCV(RandomForestClassifier(), RFParam)
RF_scores = cross_val_score(RF_Grid, train_X, train_y)
(RF_scores.mean(), RF_scores.std())

(0.7949163954662776, 0.045809452945543803)

In [107]:
RF_Grid.fit(train_X, train_y)
pred = RF_Grid.predict(test_X)
accuracy_score(test_y, pred)

0.8295964125560538

# GaussianNB

In [108]:
train_X, test_X, train_y, test_y = train_test_split(df_RF.drop('Survived', axis=1), df_RF['Survived'], random_state=42)

model = GaussianNB()

GNB_scores = cross_val_score(model, train_X, train_y)
(GNB_scores.mean(), GNB_scores.std())

(0.7260913477724161, 0.08824350567070521)

In [109]:
model.fit(train_X, train_y)
pred = model.predict(test_X)
accuracy_score(test_y, pred)

0.7757847533632287

In [110]:
GNBParam = {
    'var_smoothing': [1e-9, 1e-6, 1e-5]
}

GNB_Grid = GridSearchCV(GaussianNB(), GNBParam)
GNB_scores = cross_val_score(GNB_Grid, train_X, train_y)
(GNB_scores.mean(), GNB_scores.std())

(0.7843676355066772, 0.02530568984570613)

In [111]:
GNB_Grid.fit(train_X, train_y)
pred = GNB_Grid.predict(test_X)
accuracy_score(test_y, pred)

0.7847533632286996